In [1]:
# !pip install python-keycloak
# !pip install requests
# # !pip install BeautifulSoup

In [3]:
import requests
from bs4 import BeautifulSoup
from keycloak import KeycloakOpenID
import  urllib.parse

keycloak_openid = KeycloakOpenID(server_url="http://localhost:8080/auth/",
                                 client_id="rolap-web",
                                 realm_name="ROLAP",
                                 client_secret_key="CN4jMjmop2SdIZaEU5LW0Xla2wMM94Nt")

config_well_known = keycloak_openid.well_known()

auth_url = keycloak_openid.auth_url(
    redirect_uri="http://localhost:4200/callback",
    scope="email profile openid",
    state="your_state_info")

response = requests.get(auth_url)

soup = BeautifulSoup(response.content, "html")
action = soup.find('form').get('action')

credentials_payload = { 'username': 'alex', 'password': 'Matematyka1!'}

session = requests.Session()
session.max_redirects = 1
response2 = session.post(action, cookies=response.cookies, data=credentials_payload, allow_redirects=False)

redirect_url = response2.headers['Location']

parsed_redirect_url = urllib.parse.urlparse(redirect_url)
code = captured_value = urllib.parse.parse_qs(parsed_redirect_url.query)['code'][0]

access_token = keycloak_openid.token(
    grant_type='authorization_code',
    code=code,
    redirect_uri="http://localhost:4200/callback")

print('Bearer ' + access_token['access_token'])

Bearer eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJJendtSm1JRVk3N1EybVVwUWpReWtwWWR6Q1h3M3FzN0dyNHlIMzBIX3NjIn0.eyJleHAiOjE2ODY3NzQyNjcsImlhdCI6MTY4Njc3Mzk2NywiYXV0aF90aW1lIjoxNjg2NzczOTY3LCJqdGkiOiJhZjMxNGRkYy0zNTZmLTRjY2MtYmI5YS1iOTMxN2Q3YWYxZTkiLCJpc3MiOiJodHRwOi8vbG9jYWxob3N0OjgwODAvYXV0aC9yZWFsbXMvUk9MQVAiLCJhdWQiOiJhY2NvdW50Iiwic3ViIjoiYmVmYzVjNTUtYTM3Zi00MWU1LTk4ZmEtNmQxODE3NzgxMWY5IiwidHlwIjoiQmVhcmVyIiwiYXpwIjoicm9sYXAtd2ViIiwic2Vzc2lvbl9zdGF0ZSI6ImM5ZWI3Njg2LWM4MjEtNDA5Zi04YTg3LWVmYWRlNDM5OTUxNCIsImFjciI6IjEiLCJhbGxvd2VkLW9yaWdpbnMiOlsiKiJdLCJyZWFsbV9hY2Nlc3MiOnsicm9sZXMiOlsiZGVmYXVsdC1yb2xlcy1yb2xhcCIsIm9mZmxpbmVfYWNjZXNzIiwidW1hX2F1dGhvcml6YXRpb24iXX0sInJlc291cmNlX2FjY2VzcyI6eyJhY2NvdW50Ijp7InJvbGVzIjpbIm1hbmFnZS1hY2NvdW50IiwibWFuYWdlLWFjY291bnQtbGlua3MiLCJ2aWV3LXByb2ZpbGUiXX19LCJzY29wZSI6Im9wZW5pZCBwcm9maWxlIGVtYWlsIiwic2lkIjoiYzllYjc2ODYtYzgyMS00MDlmLThhODctZWZhZGU0Mzk5NTE0IiwiZW1haWxfdmVyaWZpZWQiOnRydWUsIm5hbWUiOiJBbGVrc2FuZHJhIExld2FuZG93c2thIiwicHJlZmVycmVkX3VzZ